In [31]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
"""
tf.function
Compiles a function into a callable TensorFlow graph.
被装饰的这个方法会被tensorflow解析，将这里的运算逻辑放入graph中
tensorflow是定义与运算分离。
可以看到下面的exec只会执行一次，这时实际是在trace阶段（追踪数据流向轨迹、定义或者说编译）

先看下面的两个f1，说明trace阶段是在第一次调用是做的
而第二次调用的时候，实际上就只执行了f1中的图计算部分，即生成一个张量以及循环的三次write

再看f2部分，可以看到l是有3个值，而且只是普通的没有值的tensor
这是因为这个f2里除了一个循环之外，并没有真正的图计算的op，因此没有读取到值

f3里只会执行一次，这也是因为除了循环之外没有真正的图计算op，图里只知道有一个循环
所以结果只是一个while/add:0
但至于为什么f2的循环会执行三次，我猜只可能是lenx和直接对x循环的差别吧
"""


@tf.function
def f1(x):
  print("exec 1")
  ta = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
  for i in range(len(x)):
    print(i)
    ta = ta.write(i, x[i] + 1)
  return ta.stack()
f1(tf.constant([1, 2, 3]))
f1(tf.constant([1, 2, 3]))

l = []
@tf.function
def f2(x):
  print("exec 1")
  for i in range(len(x)):
    print(i) # 那这个为啥会执行三次啊
    l.append(x[i])
f2(tf.constant([1, 2, 3]))
print(l)

l=[]
@tf.function
def f3(x):
  for i in x:
    l.append(i + 1)    # Caution! Will only happen once when tracing
f3(tf.constant([1, 2, 3]))
print(l)
f3(tf.constant([1, 2, 3]))
print(l)

exec 1
0
1
2
exec 1
0
1
2
[<tf.Tensor 'strided_slice:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>, <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>]
[<tf.Tensor 'while/add:0' shape=() dtype=int32>]
[<tf.Tensor 'while/add:0' shape=() dtype=int32>]


tf.Tensor(
[[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]
  [12 13 14 15 16 17]
  [18 19 20 21 22 23]]

 [[24 25 26 27 28 29]
  [30 31 32 33 34 35]
  [36 37 38 39 40 41]
  [42 43 44 45 46 47]]], shape=(2, 4, 6), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]]

  [[ 6  7]
   [ 8  9]
   [10 11]]

  [[12 13]
   [14 15]
   [16 17]]

  [[18 19]
   [20 21]
   [22 23]]]


 [[[24 25]
   [26 27]
   [28 29]]

  [[30 31]
   [32 33]
   [34 35]]

  [[36 37]
   [38 39]
   [40 41]]

  [[42 43]
   [44 45]
   [46 47]]]], shape=(2, 4, 3, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1]
   [ 2  3]
   [ 4  5]
   [ 6  7]]

  [[ 8  9]
   [10 11]
   [12 13]
   [14 15]]

  [[16 17]
   [18 19]
   [20 21]
   [22 23]]]


 [[[24 25]
   [26 27]
   [28 29]
   [30 31]]

  [[32 33]
   [34 35]
   [36 37]
   [38 39]]

  [[40 41]
   [42 43]
   [44 45]
   [46 47]]]], shape=(2, 3, 4, 2), dtype=int32)
tf.Tensor(
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]]

  [[12 13 14 15]
   [16 17 18 19]
   [20 21 22 23]]]


 [[[24 